In [1]:
import plotly.express as px
import requests
import pandas as pd
import numpy as np

In [11]:
stats = pd.read_excel("C:/Users/ikein/Documents/MSA/side projects/Sleeper-Analysis/Taylor/SleeperData.xlsx")
stats

,Owner,Week,Player,player_id,age,position,gp,off_snp,tm_off_snp,pts_ppr,...,pass_td,sacks,fum_rec,int,def_td,yds_allow,pts_allow,blk_kick,fum,fum_lost
0,keaton22,1,Cole Beasley,1144,32.0,WR,1.0,77.0,85.0,14.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,keaton22,1,Keenan Allen,1479,29.0,WR,1.0,67.0,81.0,18.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,keaton22,1,Devonta Freeman,2168,29.0,RB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,keaton22,1,Melvin Gordon,2320,28.0,RB,1.0,33.0,66.0,20.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,keaton22,1,Dak Prescott,3294,28.0,QB,1.0,83.0,83.0,27.42,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,KingDarsh,Season,Brandon Aiyuk,6803,23.0,WR,17.0,841.0,1021.0,168.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0
3276,KingDarsh,Season,James Robinson,6955,23.0,RB,14.0,530.0,909.0,169.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0
3277,KingDarsh,Season,Najee Harris,7528,23.0,RB,17.0,940.0,1097.0,300.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,KingDarsh,Season,Buffalo Bills,BUF,NaN,DEF,17.0,NaN,NaN,157.00,...,NaN,NaN,8.0,19.0,1.0,4637.0,287.0,2.0,NaN,NaN


In [63]:
age2 = stats[stats["Week"] == "Season"][["Owner", "age", "position", "Week"]]
age2 = age2.groupby(["Owner", "position"]).agg({"age":np.mean})
age2

age
Owner     position           
Conner123 DEF             NaN
          K         32.000000
          QB        44.000000
          RB        24.400000
          TE        27.333333
...                       ...
vunosh    K         24.000000
          QB        22.000000
          RB        24.333333
          TE        29.500000
          WR        27.000000

[72 rows x 1 columns]

In [56]:
age2['avg'] = age2[age2["Week"] == "Season"].groupby("Owner").agg({"age" : np.mean})


<ipython-input-56-e5ba2a57b2d4>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
age = pd.DataFrame(stats[stats["Week"] == "Season"].groupby("Owner").agg({"age": np.mean}))
age

,age
Owner,
Conner123,27.071429
KingDarsh,26.642857
Placejt,27.055556
adambloom,25.705882
chapelf,26.466667
ikei,25.875000
jfortkort5,26.058824
johale,26.066667
keaton22,27.625000


In [66]:
list(age2.columns)

['age']

In [76]:
fig = px.bar(age, y = "age", color = "age")
fig.show()

In [73]:
fig = px.bar(age2)

TypeError: Data frame index is a pandas MultiIndex. pandas MultiIndex is not supported by plotly express at the moment.

In [75]:
pts = pd.DataFrame(stats[stats["Week"] == "Season"].groupby("Owner").agg({"pts_half_ppr": np.mean}))
pts

,pts_half_ppr
Owner,
Conner123,135.352222
KingDarsh,151.372500
Placejt,143.808421
adambloom,152.684444
chapelf,151.240000
ikei,122.590588
jfortkort5,160.248889
johale,168.041250
keaton22,156.954118


In [78]:
fig = px.bar(pts, x = "pts_half_ppr", color = "pts_half_ppr")
fig.show()

In [82]:
fig = px.line(pts, x = "Owner", y = "pts_half_ppr")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['pts_half_ppr'] but received: Owner

In [83]:
fig = px.line(pts, y = "pts_half_ppr")
fig.show()

In [84]:
pts = pd.DataFrame(stats[stats["Week"] != "Season"].groupby("Owner").agg({"pts_half_ppr": np.mean}))
pts

,pts_half_ppr
Owner,
Conner123,9.724151
KingDarsh,11.019027
Placejt,9.678435
adambloom,10.664404
chapelf,10.391373
ikei,9.445668
jfortkort5,11.829538
johale,10.897500
keaton22,11.085445


In [93]:
fig = px.line(stats, x = "Week", y = "Owner", color = "pts_half_ppr")
fig.show()

KeyError: (nan, '', '', '', '', '', '')